In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np

from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

In [2]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [3]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

[2022/03/01 12:04:08] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/03/01 12:04:08] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/03/01 12:04:08] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [2]:
gui = GUI('data/input/temp.jpg')
# gui.detect_element(True, True, True, paddle_cor=paddle_ocr)
gui.load_detection_result()
# gui.show_detection_result()

In [5]:
# load operations
testing_data_root = '/home/ml/Data/visual testing/experiments/humanoid-output/adguard-1'

steps_file = pjoin(testing_data_root, 'output.json')
steps = json.load(open(steps_file, 'r'))

operations = []
for step in steps:
    op = Operation(pjoin(testing_data_root, step['tag'] + '.jpg'), step['event_type'], step['bounds'])
    op.shrink_target_element_img()
    op.resize(gui.detection_resize_width, gui.detection_resize_height)        
    op.detect_text(paddle_ocr)
    operations.append(op)
#     op.show_target_ele()
    break

[2022/03/01 10:51:41] root DEBUG: dt_boxes num : 0, elapse : 0.030437469482421875
[2022/03/01 10:51:41] root DEBUG: cls num  : 0, elapse : 0
[2022/03/01 10:51:41] root DEBUG: rec_res num  : 0, elapse : 1.1920928955078125e-06
[Text Detection Completed in 0.033 s] Input: data/operation/target.png Output: data/operation/target.json


In [10]:
opr = operations[0]
gui.match_elements(target_ele_img=opr.target_element_img, target_ele_text=opr.get_text_content(), resnet_model=resnet_model, show=True)

Match by image


In [10]:
operations[0].show_target_ele()

In [5]:
gui.adjust_elements_by_screen()

In [7]:
b = gui.draw_element_after_screen_conversion()

In [21]:
gui.screen.width

335

In [22]:
gui.screen.height

725

In [25]:
gui.screen.clip.shape

(743, 343, 3)

In [29]:
for ele in gui.elements:
    print(ele.height, ele.width, ele.clip.shape)

725 335 (743, 343, 3)
12 9 (12, 9, 3)
12 18 (12, 19, 3)
15 17 (15, 17, 3)
28 34 (29, 34, 3)
20 16 (21, 16, 3)
18 19 (18, 19, 3)
17 18 (18, 18, 3)
22 22 (23, 22, 3)
11 37 (12, 38, 3)
15 39 (16, 40, 3)
11 15 (12, 15, 3)
17 16 (18, 17, 3)
14 16 (15, 17, 3)
43 5 (44, 5, 3)
14 89 (14, 91, 3)
18 69 (18, 70, 3)
16 158 (17, 162, 3)
16 178 (16, 183, 3)
16 100 (16, 102, 3)
13 99 (13, 101, 3)
13 90 (13, 92, 3)
11 17 (12, 17, 3)
13 62 (14, 63, 3)
13 32 (14, 32, 3)
14 46 (15, 47, 3)


In [28]:
gui.load_detection_result()
gui.show_detection_result()

In [15]:
gui.recognize_phone_screen()

In [16]:
gui.remove_ele_out_screen()

In [5]:
gui.convert_element_relative_pos_by_screen()

In [24]:
b = gui.draw_element_after_screen_conversion()

In [12]:
gui.show_detection_result()

In [23]:
gui.show_detection_result()